In [1]:
import pandas as pd
import numpy as np
import nmf

udata = pd.read_table('ml-100k/u.data', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])

R = pd.DataFrame(index=range(0,943),columns=range(0,1682))

for row in udata.itertuples():
    R[row[1]-1][row[2]-1] = row[3]

R = R.fillna(0)

W = R.copy()
W = W.fillna(0)
W[W > 0] = 1

In [2]:
### Problem 1 ###
k = [10, 50, 100]

U10, V10 = nmf.nmf(R, k[0], 0)
U50, V50 = nmf.nmf(R, k[1], 0)
U100, V100 = nmf.nmf(R, k[2], 0)
# NMF_model10 = decomposition.NMF(k[0])
# NMF_model50 = decomposition.NMF(k[1])
# NMF_model100 = decomposition.NMF(k[2])

# U10 = NMF_model10.fit_transform(R, W=W)
# V10 = NMF_model10.components_
# U50 = NMF_model50.fit_transform(R, W=W)
# V50 = NMF_model50.components_
# U100 = NMF_model100.fit_transform(R, W=W)
# V100 = NMF_model100.components_

def lse(w, r, u, v):
    error = w * (r - u.dot(v))**2
    return error.sum().sum()

err10 = lse(W, R, U10, V10)
err50 = lse(W, R, U50, V50)
err100 = lse(W, R, U100, V100)

In [3]:
print("For k = %d, the least squared error is %f" % (k[0], err10))
print("For k = %d, the least squared error is %f" % (k[1], err50))
print("For k = %d, the least squared error is %f" % (k[2], err100))

For k = 10, the least squared error is 3648.317694
For k = 50, the least squared error is 83.918589
For k = 100, the least squared error is 6.234883


In [4]:
### Problem 2 ###
# k-fold
from scipy.sparse import csr_matrix
from sklearn.cross_validation import KFold
from sklearn.metrics import auc
import matplotlib.pyplot as plt

A = csr_matrix(R.values)
N = A.getnnz()
nonzero_indices = zip(A.nonzero()[0], A.nonzero()[1])

def abs_err(actual, u, v, w, size):
    error = abs(w * (u.dot(v) - actual))
    return error.sum().sum() / size

def set_values(M, rows, columns, value):
    for index in zip(rows, columns):
        M.ix[index] = value

kf = KFold(n=N, n_folds=10, shuffle=True)
errors = []
i = 1
for train_index, test_index in kf:
    ri = A.nonzero()[0][test_index]
    ci = A.nonzero()[1][test_index]
    train = R.copy()
    set_values(train, ri, ci, 0)
    w = pd.DataFrame(index=range(0,943),columns=range(0,1682))
    w = w.fillna(0)
    set_values(w, ri, ci, 1)
    test_size = len(test_index)
    u,v = nmf.nmf(train, 100, 0)
    errors.append(abs_err(R, u, v, w, test_size))
    print(errors[-1])
    
    ### Problem 3 ###
    precs = []
    recalls = []
    for threshold in range(1,6):
        print("-------------------")
        print("\tThreshold = %d" % threshold)
        test_result = pd.DataFrame(u.dot(v))
        pred = test_result.lookup(A.nonzero()[0][test_index], A.nonzero()[1][test_index])
        true = R.lookup(A.nonzero()[0][test_index], A.nonzero()[1][test_index])
        precision = 0 if np.sum(pred > threshold) == 0 \
                      else np.sum(true[pred > threshold] > threshold) / float(np.sum(pred > threshold)) * 100
        print("\tprecision: %.2f%%" % precision)
        recall = 0 if np.sum(true > threshold) == 0 \
                   else np.sum(pred[true > threshold] > threshold) / float(np.sum(true > threshold)) * 100
        print("\trecall: %.2f%%" % recall)
        precs.append(precision)
        recalls.append(recall)
    print("\tarea under curve: %.2f" % auc(recalls, precs))
    plt.cla()
    plt.xlim(xmin=0, xmax=np.max(recalls)*1.1)
    plt.ylim(ymin=0, ymax=np.max(precs)*1.1)
    plt.scatter(recalls, precs, s=60, marker='o')
    plt.plot(recalls, precs)
    plt.fill_between(recalls, precs, 0)
    plt.title("ROC Precision over Recall " + str(i))
    plt.xlabel('Recall (%)')
    plt.ylabel('Precision (%)')
    plt.savefig('graphs/problem3_ROC' + str(i))
    plt.close()
    i = i + 1

print("Highest absolute error is %f" % max(errors))
print("Lowest absolute error is %f" % min(errors))

1.09381370476
-------------------
	Threshold = 1
	precision: 95.38%
	recall: 98.63%
-------------------
	Threshold = 2
	precision: 87.97%
	recall: 92.01%
-------------------
	Threshold = 3
	precision: 68.43%
	recall: 64.92%
-------------------
	Threshold = 4
	precision: 38.51%
	recall: 30.40%
-------------------
	Threshold = 5
	precision: 0.00%
	recall: 0.00%
	area under curve: 5157.06
1.0607690476
-------------------
	Threshold = 1
	precision: 95.64%
	recall: 98.57%
-------------------
	Threshold = 2
	precision: 87.51%
	recall: 92.34%
-------------------
	Threshold = 3
	precision: 70.57%
	recall: 65.84%
-------------------
	Threshold = 4
	precision: 37.76%
	recall: 28.42%
-------------------
	Threshold = 5
	precision: 0.00%
	recall: 0.00%
	area under curve: 5228.58
1.05577798219
-------------------
	Threshold = 1
	precision: 94.21%
	recall: 98.90%
-------------------
	Threshold = 2
	precision: 87.12%
	recall: 93.58%
-------------------
	Threshold = 3
	precision: 69.60%
	recall: 70.04%

In [5]:
### Problem 4 ###
new_W = R.copy()
new_R = W.copy()

U10, V10 = nmf.nmf(new_R, k[0], 0)
U50, V50 = nmf.nmf(new_R, k[1], 0)
U100, V100 = nmf.nmf(new_R, k[2], 0)
# U10 = NMF_model10.fit_transform(new_R, W=new_W)
# V10 = NMF_model10.components_
# U50 = NMF_model50.fit_transform(new_R, W=new_W)
# V50 = NMF_model50.components_
# U100 = NMF_model100.fit_transform(new_R, W=new_W)
# V100 = NMF_model100.components_

new_err10 = lse(new_W, new_R, U10, V10)
new_err50 = lse(new_W, new_R, U50, V50)
new_err100 = lse(new_W, new_R, U100, V100)

In [6]:
print("For k = %d, the least squared error is %f" % (k[0], new_err10))
print("For k = %d, the least squared error is %f" % (k[1], new_err50))
print("For k = %d, the least squared error is %f" % (k[2], new_err100)) 
# Total squared error decreased

For k = 10, the least squared error is 2.010969
For k = 50, the least squared error is 2.733951
For k = 100, the least squared error is 0.482370


In [38]:
new_errors = []
#new_NMF_model100 = decomposition.NMF(100, alpha=l, l1_ratio=0.0)
lambdas = [0.01, 0.1, 1]

i = 1
for train_index, test_index in kf:
    ri = A.nonzero()[0][test_index]
    ci = A.nonzero()[1][test_index]
    # set 10% test ratings to 0
    train = new_R.copy()
    set_values(train, ri, ci, 0)
    # set 10% test weights to 0
    w = new_W.copy()
    set_values(w, ri, ci, 0)
    # create filter for the 10% testing samples
    testw = pd.DataFrame(index=range(0,943),columns=range(0,1682))
    testw = testw.fillna(0)
    set_values(testw, ri, ci, 1)
    test_size = len(test_index)
    # run alternating least square for each lambda value
    for l in lambdas:
        err = []
        u, v = nmf.nmf(train, 100, l)
        err.append(abs_err(new_R, u, v, testw, test_size))
        print("lambda = %f; error = %f" % (l, err[-1]))
        new_errors.append(err[-1])
    
        new_precs = []
        new_recalls = []
        for threshold in np.arange(0.0, 1.5, 0.25):
            print("-------------------")
            print("\tThreshold = %.2f" % threshold)
            test_result = pd.DataFrame(u.dot(v))
            pred = test_result.lookup(A.nonzero()[0][test_index], A.nonzero()[1][test_index])
            true = new_R.lookup(A.nonzero()[0][test_index], A.nonzero()[1][test_index])
            precision = 0 if np.sum(pred > threshold) == 0 \
                          else np.sum(true[pred > threshold] > threshold) / float(np.sum(pred > threshold)) * 100
            print("\tprecision: %.2f%%" % precision)
            recall = 0 if np.sum(true > threshold) == 0 \
                       else np.sum(pred[true > threshold] > threshold) / float(np.sum(true > threshold)) * 100
            print("\trecall: %.2f%%" % recall) 
            new_precs.append(precision)
            new_recalls.append(recall)
        print("\tarea under curve: %.2f" % auc(new_recalls, new_precs))
        plt.cla()
        plt.xlim(xmin=0, xmax=np.max(new_recalls)*1.1)
        plt.ylim(ymin=0, ymax=np.max(new_precs)*1.1)
        plt.scatter(new_recalls, new_precs, s=60, marker='o')
        plt.plot(new_recalls, new_precs)
        plt.fill_between(new_recalls, new_precs, 0)
        plt.title("new ROC Precision over Recall " + str(i) + 'lambda' + str(l).replace('.','_'))
        plt.xlabel('Recall (%)')
        plt.ylabel('Precision (%)')
        plt.savefig('graphs/problem4_ROC' + str(i) + 'lambda' + str(l).replace('.','_'))
        plt.close()
    i = i + 1

print("Highest absolute error is %f" % max(new_errors))
print("Lowest absolute error is %f" % min(new_errors))

lambda = 0.010000; error = 8.993032
-------------------
	Threshold = 0.00
	precision: 100.00%
	recall: 100.00%
-------------------
	Threshold = 0.25
	precision: 100.00%
	recall: 98.38%
-------------------
	Threshold = 0.50
	precision: 100.00%
	recall: 95.53%
-------------------
	Threshold = 0.75
	precision: 100.00%
	recall: 82.37%
-------------------
	Threshold = 1.00
	precision: 0.00%
	recall: 0.00%
-------------------
	Threshold = 1.25
	precision: 0.00%
	recall: 0.00%
	area under curve: 5881.40
lambda = 0.100000; error = 8.929282
-------------------
	Threshold = 0.00
	precision: 100.00%
	recall: 100.00%
-------------------
	Threshold = 0.25
	precision: 100.00%
	recall: 98.57%
-------------------
	Threshold = 0.50
	precision: 100.00%
	recall: 96.63%
-------------------
	Threshold = 0.75
	precision: 100.00%
	recall: 86.78%
-------------------
	Threshold = 1.00
	precision: 0.00%
	recall: 0.00%
-------------------
	Threshold = 1.25
	precision: 0.00%
	recall: 0.00%
	area under curve: 5661

In [43]:
### Problem 5 ###
